Import needed modules

In [11]:
from pymonntorch import *
from matplotlib import pyplot as plt
import numpy as np
from timeRes import TimeResolution
from current import *
from models import *

### LIF model

$$\tau\frac{du}{dt}= -(u-u_{rest})+R.I(t)$$

### ELIF model

$$\tau\frac{du}{dt}= -(u-u_{rest})+\Delta_{t} exp(\frac{u-\theta_{rh}}{\Delta_{t}})+R.I(t)$$

### AELIF model

$$\tau_{m}\frac{du}{dt}= -(u-u_{rest})+\Delta_{t} exp(\frac{u-\theta_{rh}}{\Delta_{t}})-Rw+R.I(t)$$
$$\tau_{w}\frac{dw}{dt}= a(u-u_{rest})-w+b\tau_{w}\sum_{t^{f}}^{}\delta(t-t^{f})$$

Analyzing models functionalities with different currents as stimuli

constant current

LIF

lif_net = Network(behavior={1: TimeResolution()}, dtype=torch.float64)